[View in Colaboratory](https://colab.research.google.com/github/kintarou/django-starter/blob/master/neural_network_iris.ipynb)

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
%precision 3

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


'%.3f'

In [0]:
from sklearn.neural_network import MLPClassifier

In [0]:
iris = load_iris()

In [4]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [5]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [6]:
X = iris.data[50:150, 0:2]
y = iris.target[50:150]
print("説明変数の行数・列数:", X.shape)

説明変数の行数・列数: (100, 2)


In [7]:
print("応答変数の行数・列数:", y.shape)

応答変数の行数・列数: (100,)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)
print("説明変数の行数・列数:", X_train.shape)
print("応答変数の行数・列数:", y_train.shape)

説明変数の行数・列数: (75, 2)
応答変数の行数・列数: (75,)


In [9]:
y_train[0:10]

array([1, 1, 2, 2, 2, 2, 1, 1, 1, 1])

In [10]:
X_train_df = pd.DataFrame(X_train, columns = ["sepal_len","sepal_wid"])
y_train_df = pd.DataFrame({"species":y_train - 1})
iris_train_df = pd.concat([y_train_df,X_train_df], axis=1)
print(iris_train_df.head(3))

   species  sepal_len  sepal_wid
0        0        5.7        2.8
1        0        6.6        3.0
2        1        6.1        3.0


In [11]:
logi_mod_full = smf.glm("species~sepal_len + sepal_wid", data = iris_train_df, family = sm.families.Binomial()).fit()
logi_mod_len = smf.glm("species~sepal_len + sepal_wid", data = iris_train_df, family = sm.families.Binomial()).fit()
logi_mod_wid = smf.glm("species~sepal_wid", data = iris_train_df, family = sm.families.Binomial()).fit()
logi_mod_null = smf.glm("species~1 + sepal_wid", data = iris_train_df, family = sm.families.Binomial()).fit()
print("full", logi_mod_full.aic.round(3))
print("len", logi_mod_len.aic.round(3))
print("wid", logi_mod_wid.aic.round(3))
print("null", logi_mod_null.aic.round(3))

full 76.813
len 76.813
wid 92.768
null 92.768


In [12]:
logi_mod_len.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-17.8358,4.232,-4.215,0.000,-26.130,-9.542
sepal_len,2.3211,0.678,3.424,0.001,0.992,3.650
sepal_wid,1.2006,1.026,1.170,0.242,-0.810,3.212


In [13]:
X_test_df = pd.DataFrame(X_test, columns = ["sepal_len", "sepal_wid"])
logi_fit = logi_mod_len.fittedvalues.round(0)
logi_pred = logi_mod_len.predict(X_test_df).round(0)
true_train = sp.sum(logi_fit == (y_train - 1))
true_test = sp.sum(logi_pred == (y_test - 1))
result_train = true_train / len(y_train)
result_test = true_test / len(y_test)
print("訓練データの的中率    :", result_train)
print("テストデータの的中率  :", result_test)
                   

訓練データの的中率    : 0.7733333333333333
テストデータの的中率  : 0.6


In [0]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
sp.std(X_train_scaled, axis = 0)

array([1., 1.])

In [16]:
sp.std(X_test_scaled, axis = 0)


array([0.74 , 0.679])

In [17]:
nnet = MLPClassifier(
    hidden_layer_sizes = (100, 100),
    alpha = 0.07,
    max_iter = 10000,
    random_state = 0)
nnet.fit(X_train_scaled, y_train)
print("訓練データの的中率　:", nnet.score(X_train_scaled, y_train))
print("テストデータの的中率:", nnet.score(X_test_scaled, y_test))

訓練データの的中率　: 0.8933333333333333
テストデータの的中率: 0.68
